## Title

**Authors:** <br>
Björn Hahn () <br>
Domenico Ferraro ()<br>
Justus Raabe (940919) <br>
Lisa Sachau (941367)

#### Imported Packages

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
import functions as func


#### Load Data

In [ ]:
# Get file names
folder_e = './Fotos_einfach'
folder_h = './Fotos_schwierig'
#image_names, categories = read_image_names(filepath)
categories_e, image_names_e, labels_e = func.get_filenames_labels(folder_e)
categories_h, image_names_h, labels_h = func.get_filenames_labels(folder_h)
print(image_names_e)
print(labels_e)
print(categories_e)
print(image_names_h)
print(labels_h)
print(categories_h)
# Load images
input_shape = {
    "height": 200,
    "width": 200,
    "channels": 3
}
images_e = func.read_images(image_names_e, height=input_shape["height"], width=input_shape["width"])
images_h = func.read_images(image_names_h, height=input_shape["height"], width=input_shape["width"])
# Convert images to array
aimages_e = func.images_to_array(images_e)
aimages_h = func.images_to_array(images_h)
print(f'Shape of images array: {aimages_e.shape}')
# Plot images array
fig = plt.figure(figsize=(8, 8))
rows, columns = 8, 10
for i in range(aimages_h.shape[0]):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(aimages_h[i, ...])
plt.show()


#### Data Augmentation

In [ ]:
def augment_images(images, labels):
    aug_images = []
    aug_labels = []
    for i, row in enumerate(images):
        # Original
        aug_images.append(images[i])
        aug_labels.append(labels[i])
        
        # 90-degree rotation (to the right)
        aug_images.append(np.rot90(row, k=1))
        aug_labels.append(labels[i])
        
        # 180-degree rotation (upside down)
        aug_images.append(np.rot90(row, k=2))
        aug_labels.append(labels[i])
        
        # 270-degree rotation (to the left)
        aug_images.append(np.rot90(row, k=3))
        aug_labels.append(labels[i])
    
    return aug_images, aug_labels

# Apply augmentation to easy images
a_images_e, a_labels_e = augment_images(aimages_e, labels_e)
aug_images_e = func.images_to_array(a_images_e)
aug_labels_e = func.images_to_array(a_labels_e)

# Apply augmentation to difficult images
a_images_h, a_labels_h = augment_images(aimages_h, labels_h)
aug_images_h = func.images_to_array(a_images_h)
aug_labels_h = func.images_to_array(a_labels_h)

# Visualize the augmented images with dynamic grid
total_images = aug_images_h.shape[0]
columns = 10
rows = int(np.ceil(total_images / columns))  # Dynamically calculate the number of rows

fig = plt.figure(figsize=(8, 8))
for i in range(total_images):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(aug_images_h[i, ...])
    plt.axis('off')  # Hide axes
plt.tight_layout()
plt.show()

# Print augmented labels and shape
print(aug_labels_h)
print(aug_labels_h.shape)
print(categories_h)


Encode Labels

In [ ]:
y_e = func.ownOneHotEncoder(aug_labels_e, categories_e)
y_h = func.ownOneHotEncoder(aug_labels_h, categories_h)

Training and Test-Split with K-Fold (vielleicht K-Fold auch später, unsicher bei Reihenfolge tbh)

In [ ]:
data_e = aug_images_e
data_h = aug_images_h
k = 6
image_folds_e, y_folds_e = func.stratified_k_fold(data_e, y_e, k)
image_folds_h, y_folds_h = func.stratified_k_fold(data_h, y_h, k)
fig = plt.figure(figsize=(2, 1))
plt.imshow(image_folds_e[3][2])
plt.imshow(image_folds_h[3][2])
plt.show()
print(y_folds_e[3][2])
print(y_folds_h[3][2])

#### Build Model

easy difficulty

In [ ]:
epoch = 100
eval_accuracy_e = []
accuracy_e = []
z = list(range(k)) 
for i in range(k):
    test_fold_index = z[k-1] if k % 2 == 0 else z[k-2]
    if test_fold_index == i:
        continue
    fold_x_train = np.concatenate([image_folds_e[j] for j in range(k) if j != i and j != test_fold_index], axis=0)
    fold_y_train = np.concatenate([y_folds_e[j] for j in range(k) if j != i and j != test_fold_index], axis=0)
    fold_x_val = image_folds_e[i]
    fold_y_val = y_folds_e[i]
    fold_x_test = image_folds_e[test_fold_index]
    fold_y_test = y_folds_e[test_fold_index]
    model, model_accuracy = func.build_model(epoch, fold_x_train, fold_y_train, fold_x_val, fold_y_val, fold_x_test, fold_y_test, input_shape)
    eval_accuracy_e.append(model_accuracy)
    accuracy_e.append(max(model.history['accuracy']))
    z = z[:-1] 
    z = [test_fold_index] + z
    
print(eval_accuracy_e)
print(accuracy_e)

Plotting of Accuracy and Loss

In [ ]:
# noch zu überarbeiten

plt.plot(range(1, k+1), accuracy_e)
plt.plot(range(1, k+1), eval_accuracy_e)
plt.title('Accuracy und Accuracy des Modells')
plt.xlabel('Epochen')
plt.gca().legend(('Accuracy', 'Accuracy des Modells'))
plt.show()

hard difficulty

In [ ]:
epoch = 20
eval_accuracy = []
accuracy = []
for i in range(k):
    fold_x_train = np.concatenate([image_folds_h[j] for j in range(k) if j != i], axis=0)
    fold_y_train = np.concatenate([y_folds_h[j] for j in range(k) if j != i], axis=0)
    fold_x_val = image_folds_h[i]
    fold_y_val = y_folds_h[i]
    model, model_accuracy = func.build_model(epoch, fold_x_train, fold_y_train, fold_x_val, fold_y_val, input_shape, y_h)
    eval_accuracy.append(model_accuracy)
    accuracy.append(max(model.history['accuracy']))
print(eval_accuracy)
print(accuracy)

Plotting of Accuracy and Loss for hard difficulty

In [ ]:
#auch noch zu überarbeiten
plt.plot(range(1, epoch+1), func.minmax(model.history['accuracy']))
plt.plot(range(1, epoch+1), func.minmax(model.history['loss']))
plt.title('Accuracy und Accuracy des Modells')
plt.xlabel('Epochen')
plt.gca().legend(('Accuracy', 'Accuracy des Modells'))
plt.show()

Prediction

In [ ]:
#prediction here